## Loading the data
Si hay multiwords, se debe saltar una linea y coger las dos siguientes. Ejemplo:

19-20	don't	_	_	_	_	_	_	_	_

19	do	do	AUX	VBP	Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbForm=Fin	21	aux	21:aux	_

20	n't	not	PART	RB	Polarity=Neg	21	advmod	21:advmod	_

In [35]:
def load_conllu_data(filepath):
    """
    Carga y procesa un archivo CoNLL-U, extrayendo las oraciones y sus etiquetas UPOS.
    """
    sentences = []
    tags = []
    current_sentence = []
    current_tags = []

    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()

            # 1. Ignorar comentarios y líneas vacías que no sean separadores de oración
            if line.startswith('#'):
                continue
            
            # 2. Línea en blanco: indica el final de una oración
            elif line == '':
                if current_sentence:
                    sentences.append(current_sentence)
                    tags.append(current_tags)
                    current_sentence = []
                    current_tags = []
            
            # 3. Procesar línea de palabra
            else:
                fields = line.split('\t')
                
                # Ignorar tokens multiword (ID con guion, e.g., '1-2') o nodos vacíos (ID con punto, e.g., '1.1') 
                if '-' in fields[0] or '.' in fields[0]:
                    continue

                # Extraer la palabra (FORM, índice 1) y la etiqueta PoS (UPOS, índice 3)
                word = fields[1]
                pos_tag = fields[3]
                
                current_sentence.append(word)
                current_tags.append(pos_tag)

    # Asegurarse de añadir la última oración si el archivo no termina en línea vacía
    if current_sentence:
        sentences.append(current_sentence)
        tags.append(current_tags)

    return sentences, tags

In [36]:
 
train_sents, train_tags = load_conllu_data('en_ewt-ud-train.conllu')
dev_sents, dev_tags = load_conllu_data('en_ewt-ud-dev.conllu')
test_sents, test_tags = load_conllu_data('en_ewt-ud-test.conllu')

print(test_sents[5], test_tags[5])

['Google', 'is', 'a', 'nice', 'search', 'engine', '.'] ['PROPN', 'AUX', 'DET', 'ADJ', 'NOUN', 'NOUN', 'PUNCT']


## 2. Text Vectorization: Creating the Dictionaries

The first step in preparing the data for the LSTM model is to convert our text-based sentences and tags into numerical sequences. Neural networks can only process numbers, so we need a consistent way to map each word and each tag to a unique integer ID.

For this task, we'll use Keras's modern `TextVectorization` layer. We will create two separate instances of this layer: one for the input words (`word_vectorizer`) and one for the output tags (`tag_vectorizer`).

The process involves two main stages:
1.  **Configuration**: We initialize the `TextVectorization` layer with `output_mode='int'` to ensure it produces sequences of integer IDs (e.g., "Google is nice" -> `[2, 3, 42]`). We also set `output_sequence_length=128` to enforce that all sequences are padded or truncated to a fixed length, which is a requirement for the model.
2.  **Adaptation**: We then call the `.adapt()` method on our training data. This step builds the internal vocabulary for each vectorizer. It analyzes all the words (or tags) in the training set and assigns a unique integer to each one. This ensures our "dictionaries" are based only on the data the model is allowed to learn from.


In [47]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

def create_and_adapt_vectorizer(sentences, max_len=128, standardize='lower_and_strip_punctuation'):

    vectorizer = TextVectorization(
        output_mode='int',
        output_sequence_length=max_len,
        standardize=standardize
    )

    sentences_flat = [' '.join(sentence) for sentence in sentences]
    
    vectorizer.adapt(sentences_flat)
    
    vocab_size = len(vectorizer.get_vocabulary())
    
    print(f"Adaptation complete. Vocabulary size: {vocab_size}")
    
    return vectorizer, vocab_size


# 2. Pasa 'standardize=None' al vectorizador de palabras
word_vectorizer, WORD_VOCAB_SIZE = create_and_adapt_vectorizer(train_sents, standardize=None)

# 3. El vectorizador de etiquetas déjalo como estaba (no necesita el cambio)
tags_vectorizer, TAGS_VOCAB_SIZE = create_and_adapt_vectorizer(train_tags)

print(f"\nWe have successfully created a vectorizer with a vocabulary of {WORD_VOCAB_SIZE} words.")

print(f"\nWe have successfully created a vectorizer with a vocabulary of {TAGS_VOCAB_SIZE} tags.")

print("Vectorizing all data sets...")

# --- 1. Flatten the data from list of lists to list of strings ---
# The vectorizer layers expect a flat list of strings as input.
train_sents_flat = [' '.join(sentence) for sentence in train_sents]
train_tags_flat = [' '.join(tag_list) for tag_list in train_tags]

dev_sents_flat = [' '.join(sentence) for sentence in dev_sents]
dev_tags_flat = [' '.join(tag_list) for tag_list in dev_tags]

test_sents_flat = [' '.join(sentence) for sentence in test_sents]
test_tags_flat = [' '.join(tag_list) for tag_list in test_tags]


# --- 2. Use the vectorizers to transform the flattened data ---
# Now we call the vectorizers with the correct input format.
X_train = word_vectorizer(train_sents_flat)
y_train = tags_vectorizer(train_tags_flat)

X_dev = word_vectorizer(dev_sents_flat)
y_dev = tags_vectorizer(dev_tags_flat)

X_test = word_vectorizer(test_sents_flat)
y_test = tags_vectorizer(test_tags_flat)

print("Vectorization complete!")
print("\nShape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_dev:", X_dev.shape)
print("Shape of y_dev:", y_dev.shape)


print(X_train[0])
print(y_train[0])
print(TAGS_VOCAB_SIZE)

Adaptation complete. Vocabulary size: 19676
Adaptation complete. Vocabulary size: 19

We have successfully created a vectorizer with a vocabulary of 19676 words.

We have successfully created a vectorizer with a vocabulary of 19 tags.
Vectorizing all data sets...
Vectorization complete!

Shape of X_train: (12544, 128)
Shape of y_train: (12544, 128)
Shape of X_dev: (2001, 128)
Shape of y_dev: (2001, 128)
tf.Tensor(
[  263    16  6153    45   293   709  1150  4859 18887   592    16 18793
     4     3  7336    35     3  7510    10     3   486     8  8931     4
   751     3  2355  1642     2     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     

## Esto es el modeo

In [48]:
import tensorflow as tf
# Importamos las capas extra que necesitamos: Bidirectional y Dropout
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, TimeDistributed, Bidirectional, Dropout
from tensorflow.keras.models import Model

EMBEDDING_DIM = 64
LSTM_UNITS = 64
DROPOUT_RATE = 0.3


# 1. Definir la capa de Entrada
inputs = Input(shape=(MAX_LEN,), name='word_ids_input')

# 2. Conectar las capas
# La capa Embedding recibe los 'inputs'
x = Embedding(
    input_dim=WORD_VOCAB_SIZE, 
    output_dim=EMBEDDING_DIM, 
    mask_zero=True, 
    name='word_embedding'
)(inputs)

# Añadimos Dropout después del embedding
x = Dropout(DROPOUT_RATE, name='dropout_embedding')(x)


# 3. Primera capa Bidirectional LSTM
x = Bidirectional(
    LSTM(
        units=LSTM_UNITS, 
        return_sequences=True,
        name='bidirectional_lstm_1'
    )
)(x)

# Añadimos Dropout después de la primera capa Bi-LSTM
x = Dropout(DROPOUT_RATE, name='dropout_lstm_1')(x)

# 4. Segunda capa Bidirectional LSTM
x = Bidirectional(
    LSTM(
        units=LSTM_UNITS, 
        return_sequences=True,
        name='bidirectional_lstm_2'
    )
)(x)

# Dropout después de la segunda capa Bi-LSTM
x = Dropout(DROPOUT_RATE, name='dropout_lstm_2')(x)


# 5. Capa de Salida (igual que antes)
outputs = TimeDistributed(
    Dense(units=TAGS_VOCAB_SIZE, activation='softmax'), 
    name='pos_tag_output'
)(x)

# 6. Crear el Modelo final
model = Model(inputs=inputs, outputs=outputs, name='pos_tagger_model_v2_bidirectional')

# ¡Listo! Ahora puedes imprimir el resumen y ver la nueva arquitectura
model.summary()

Model: "pos_tagger_model_v2_bidirectional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ word_ids_input      │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ word_embedding      │ (None, 128, 64)   │  1,259,264 │ word_ids_input[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_embedding   │ (None, 128, 64)   │          0 │ word_embedding[0… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_7         │ (None, 128)       │          0 │ word_ids_input[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_12    │ (None, 128, 128)  │     66,048 │ dropout_embeddin… │
│ (Bidirectional)     │                   │            │ not_equal_7[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_lstm_1      │ (None, 128, 128)  │          0 │ bidirectional_12… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_13    │ (None, 128, 128)  │     98,816 │ dropout_lstm_1[0… │
│ (Bidirectional)     │                   │            │ not_equal_7[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_lstm_2      │ (None, 128, 128)  │          0 │ bidirectional_13… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pos_tag_output      │ (None, 128, 19)   │      2,451 │ dropout_lstm_2[0… │
│ (TimeDistributed)   │                   │            │ not_equal_7[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,426,579 (5.44 MB)

 Trainable params: 1,426,579 (5.44 MB)

 Non-trainable params: 0 (0.00 B)

In [49]:
import tensorflow as tf
from tensorflow.keras.metrics import sparse_categorical_accuracy

def custom_masked_accuracy(y_true, y_pred):
    # 1. Crea una máscara que es 0.0 para padding (ID 0) y 1.0 para todo lo demás
    sample_weight = tf.cast(tf.not_equal(y_true, 0), dtype=tf.float32)

    # 2. Calcula la precisión para todos los tokens (incluyendo padding)
    accuracy_all_tokens = sparse_categorical_accuracy(y_true, y_pred)
    
    # 3. Aplica la máscara: multiplica la precisión de cada token por 1.0 o 0.0
    weighted_accuracy = accuracy_all_tokens * sample_weight
    
    # 4. Calcula la media solo sobre los tokens reales
    epsilon = 1e-7
    return tf.reduce_sum(weighted_accuracy) / (tf.reduce_sum(sample_weight) + epsilon)


# Compilar el modelo
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=[custom_masked_accuracy]
)

In [50]:
# 1. Importa el Callback
from tensorflow.keras.callbacks import EarlyStopping

# 2. Crea una instancia del callback
early_stopping_callback_acc = EarlyStopping(
    monitor='val_custom_masked_accuracy', 
    patience=5,
    min_delta=0.001,          # Una mejora de 0.1% 
    mode='max',               # Queremos maximizar la precisión (accuracy)
    restore_best_weights=True
)

# 3. Pasa el callback a model.fit()

# Parámetros de entrenamiento
EPOCHS = 50
BATCH_SIZE = 64

# Entrenar el modelo
history = model.fit(
    X_train, 
    y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_dev, y_dev),
    callbacks=[early_stopping_callback_acc] 
)

Epoch 1/50
196/196 ━━━━━━━━━━━━━━━━━━━━ 32s 138ms/step - custom_masked_accuracy: 0.4365 - loss: 1.7990 - val_custom_masked_accuracy: 0.7873 - val_loss: 0.7450
Epoch 2/50
196/196 ━━━━━━━━━━━━━━━━━━━━ 26s 134ms/step - custom_masked_accuracy: 0.8633 - loss: 0.4844 - val_custom_masked_accuracy: 0.8864 - val_loss: 0.3971
Epoch 3/50
196/196 ━━━━━━━━━━━━━━━━━━━━ 26s 133ms/step - custom_masked_accuracy: 0.9293 - loss: 0.2626 - val_custom_masked_accuracy: 0.8938 - val_loss: 0.3627
Epoch 4/50
196/196 ━━━━━━━━━━━━━━━━━━━━ 26s 133ms/step - custom_masked_accuracy: 0.9490 - loss: 0.1899 - val_custom_masked_accuracy: 0.8963 - val_loss: 0.3728
Epoch 5/50
196/196 ━━━━━━━━━━━━━━━━━━━━ 26s 134ms/step - custom_masked_accuracy: 0.9568 - loss: 0.1573 - val_custom_masked_accuracy: 0.9024 - val_loss: 0.3563
Epoch 6/50
196/196 ━━━━━━━━━━━━━━━━━━━━ 26s 131ms/step - custom_masked_accuracy: 0.9625 - loss: 0.1355 - val_custom_masked_accuracy: 0.9018 - val_loss: 0.3661
Epoch 7/50
196/196 ━━━━━━━━━━━━━━━━━━━━ 26s 13

In [15]:
import numpy as np


# 1. Obtenemos el vocabulario de etiquetas (la lista de strings)
# El ID 0 es '', el ID 1 es '[UNK]', así que el vocabulario real empieza en el índice 2
tags_vocab = tags_vectorizer.get_vocabulary()

def predict_tags(sentence_string):
    """
    Toma una frase (string), la procesa con el modelo 
    y muestra las predicciones palabra por palabra.
    """
    
    print(f"--- Prediciendo para: ---\n{sentence_string}\n")
    
    # 0. Separamos las palabras para saber cuántas son
    words = sentence_string.split(' ')
    
    # 1. Convertir la frase (string) en un tensor de IDs (shape [1, 128])
    # El vectorizador espera una lista de strings
    input_tensor = word_vectorizer([sentence_string])
    
    # 2. Obtener las predicciones del modelo
    # El modelo devuelve probabilidades (shape [1, 128, 20])
    predictions = model.predict(input_tensor)
    
    # 3. Encontrar el ID de la etiqueta con mayor probabilidad para cada token
    # Usamos np.argmax para obtener los IDs ganadores (shape [1, 128])
    predicted_ids = np.argmax(predictions, axis=-1)[0] # [0] para coger el primer (y único) batch
    
    # 4. Mostrar los resultados
    print("--- Resultados: ---")
    for i in range(len(words)):
        word = words[i]
        tag_id = predicted_ids[i]
        tag_name = tags_vocab[tag_id]
        
        print(f"{word:<15} -> {tag_name}")

In [20]:
# Prueba 1: La frase de ejemplo de la práctica
predict_tags("Google is a nice search engine .")

# Prueba 2: Una frase nueva
predict_tags("I am writing this new sentence for the test .")

# Prueba 3: Otra más
predict_tags("The university is in ntiago .")

--- Prediciendo para: ---
Google is a nice search engine .

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 945ms/step
--- Resultados: ---
Google          -> propn
is              -> aux
a               -> det
nice            -> adj
search          -> noun
engine          -> noun
.               -> noun
--- Prediciendo para: ---
I am writing this new sentence for the test .

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
--- Resultados: ---
I               -> pron
am              -> aux
writing         -> verb
this            -> det
new             -> adj
sentence        -> noun
for             -> adp
the             -> det
test            -> noun
.               -> noun
--- Prediciendo para: ---
The university is in ntiago .

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
--- Resultados: ---
The             -> det
university      -> propn
is              -> aux
in              -> adp
ntiago          -> propn
.               -> propn
